# Running histocytometry of DC images from Cellpose 2.0 using a script

Use cellpose2 conda environment

In [1]:
import numpy as np
import time, os, sys
from urllib.parse import urlparse
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['figure.dpi'] = 300
import cellpose 
from cellpose import utils, io, models
import re
import tifffile
from deco import concurrent, synchronized

/stor/home/jfm2773/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Functions

In [13]:
@concurrent
def segment_DCs_w_cellpose(img_dir, 
                           user_model, 
                           img_name               = "reordered_image.ome.tif", 
                           cellpose2_channels     = [2,1], 
                           DAPI_membrane_channels = [0,2]) -> None:
    img     = io.imread(os.path.join(img_dir, img_name))
    sub_img = img[DAPI_membrane_channels,...]    
        ## Subsetting image to get the membrane and DAPI channel
        ## DAPI corresponds to red (1),
        ## CD11c corresponds to green (2) 
    cellpose2_channels = [2,1]
    masks, flows, styles = user_model.eval(sub_img, channels= cellpose2_channels)
    print("Done with modeling!")

    tifffile.imwrite(os.path.join(img_dir, "cp2_mask.tif"), masks)

    ## Do I want to make the overlaid mask a binary mask?
    masks[masks > 0] = 1
    masks.astype("uint8")
    masks = masks[np.newaxis, ...]
    img_w_cp2_masks = np.concatenate((img, masks))
    tifffile.imwrite(os.path.join(img_dir, "reordered_image_w_cp2_mask.ome.tif"), img_w_cp2_masks)
    print("Done saving images!")

    return None

@synchronized 
def parallelize_segment_DCs(img_dirs, 
                            user_model, 
                            img_name               = "reordered_image.ome.tif", 
                            cellpose2_channels     = [2,1], 
                            DAPI_membrane_channels = [0,2]) -> None: 
    
    for img_dir in img_dirs:
        segment_DCs_w_cellpose(
            img_dir                = img_dir, 
            user_model             = user_model
        )
        
    return None

## DC Segmentation

In [14]:
## Load CD11c+ images
raw_dir = "/stor/scratch/Ehrlich/Users/John/histocytometry/raw_images/images_2023-08-10"
CD11c_dirs = [os.path.join(raw_dir, img_dir) for img_dir in os.listdir(raw_dir) if re.search("_[A-D]$", img_dir) and not re.search("MerTK", img_dir)]
img_name = "reordered_image.ome.tif"

In [15]:
## Load-in user trained model
model_path = "/stor/scratch/Ehrlich/Users/John/histocytometry/raw_images/images_2023-08-10/TN2_DAPI_CD11c_stringent_v1_training_images/models/TN2_DAPI_CD11c_stringent_v7"
user_model = models.CellposeModel(pretrained_model= model_path)

In [10]:
img_dir = CD11c_dirs[0]

In [11]:
img_name = "reordered_image.ome.tif"
cellpose2_channels     = [2,1]
DAPI_membrane_channels = [0,2]

In [12]:
img = io.imread(os.path.join(img_dir, img_name))
print(img.shape)
print(type(DAPI_membrane_channels))
sub_img = img[DAPI_membrane_channels,...]    

(4, 17454, 13103)
<class 'list'>


In [16]:
parallelize_segment_DCs(
    img_dirs   = CD11c_dirs, 
    user_model = user_model
) 

KeyboardInterrupt: 

In [ ]:
for img_dir in img_dirs:

    img = io.imread(os.path.join(img_dir, img_name))
    sub_img = img[[0,2],...]    
        ## Subsetting image to get the membrane and DAPI channel
        ## DAPI corresponds to red (1),
        ## CD11c corresponds to green (2) 
            ## I don't love this representation, but this is how they've hardcoded it. 
    cellpose2_channels = [2,1]
    masks, flows, styles, diams = user_model.eval(sub_img, channels= cellpose2_channels)
        ## The diaemeter will be loaded from the custom model.
        ## Channel 2 is the green representing CD11c (or CD63)
        ## Channel 1 is the red representing DAPI

    # save results so you can load in gui
    io.masks_flows_to_seg(img, masks, flows, diams, os.path.join(img_dir, "reordered_image_gui_mask"), cellpose2_channels)

    # save results 
    io.save_masks(
        images     = img, 
        masks      = masks, 
        flows      = flows, 
        file_names = os.path.join(img_dir, "save_masks_output.tif"), 
        tif        = True
    )
    
    ## 
    cellpose.plot.outline_view(img0, maski, color=[1, 0, 0], mode='inner')

In [ ]:
print(type(masks))
print(masks.dtype)
print(masks.shape)

In [3]:
# plot image with masks overlaid
mask_RGB = plot.mask_overlay(dat['img'], dat['masks'],
                        colors=np.array(dat['colors']))

## I could co-opt this code to plot the cell types on images if I replace the dat['colors'] with the corresponding cell type to each cell. 

# plot image with outlines overlaid in red
segmentation_outline = cellpose.plot.outline_view(img0, maski, color=[1, 0, 0], mode='inner')
    ## I don't think this does quite what I want. 
    ## I should really just find how to get the 2D mask then export that as its own tif then add it to reordred_image.tif

NameError: name 'plot' is not defined